Testing Notebook 

In [52]:
print("Hello World")

Hello World


The file or folder I want to test and where I want to write it. You could do this as a loop but it risks out of memory errors so I did each attack one by one.
There were also some issues with a couple of the files which would have made a loop a poor choice. These are listed at the end of this notebook.

In [53]:
analysisFile="Data/awid/CSV/12.Evil_Twin"
writePath="Data/parquet/12.Evil_Twin"

The most helpful website for me making this note book is https://sparkbyexamples.com/pyspark-tutorial/
Import packages and the data Types we need

In [54]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

see if there is an active session, there should not be, so no output when cell is run as it is not set up yet

In [55]:
SparkSession.getActiveSession()

create the session, I set the driver memory to 8gb here as I had some out of memory errors, but the settings would depend on the machine or cluster you were using. I used local mode in my project, mainly because I found the time spent spinning up my mini home cluseter was not worth it while working on the project in the evening after work.  

In [56]:
spark = SparkSession.builder \
    .master("local") \
    .appName("WiFi") \
    .config("spark.driver.memory", "8g")\
    .getOrCreate()

Schema is based on data type but also there are some excel errors in the data set so being broad to prevent data loss (there is the odd date where it should have been and some 1 values have become 1.0). Also The Vector Assembler we use in MLib turns all the ints into doubles anyway when we come to analysis. 
Spark does not like . so we use _ so we dont need to use ' (backticks) around the struts. 

Using strings slows down the analysis so favouring doubles.
Another issue I had was some particularly the TCP were numeric, but had some multiple observations in the same cell such as 01-01-01 instead of the usual 1, or more problematically "693411408-1562281087-4292509808" instead of a single value. 

I tried coding these as strings to keep the infomation, but the encoder made an enormous vector, as each reading was unique which was unworkable with. Instead I changed them to numberic values and accepted the data loss. My exploratory analysis had shown this infomation to not be important factors

In [57]:
schema = StructType() \
	.add("frame_encap_type",IntegerType(),False) \
	.add("frame_len",IntegerType(),False) \
	.add("frame_number",IntegerType(),False) \
	.add("frame_time",StringType(),False) \
	.add("frame_time_delta",DoubleType(),False) \
	.add("frame_time_delta_displayed",DoubleType(),False) \
	.add("frame_time_epoch",DoubleType(),False) \
	.add("frame_time_relative",DoubleType(),False) \
	.add("radiotap_channel_flags_cck",IntegerType(),True) \
	.add("radiotap_channel_flags_ofdm",IntegerType(),True) \
	.add("radiotap_channel_freq",IntegerType(),True) \
	.add("radiotap_datarate",DoubleType(),True) \
	.add("radiotap_dbm_antsignal",IntegerType(),True) \
	.add("radiotap_length",IntegerType(),True) \
	.add("radiotap_mactime",DoubleType(),True) \
	.add("radiotap_present_tsft",StringType(),True) \
	.add("radiotap_rxflags",StringType(),True) \
	.add("radiotap_timestamp_ts",LongType(),True) \
	.add("radiotap_vendor_oui",IntegerType(),True) \
	.add("wlan_duration",IntegerType(),False) \
	.add("wlan_analysis_kck",StringType(),True) \
	.add("wlan_analysis_kek",StringType(),True) \
	.add("wlan_bssid",StringType(),True) \
	.add("wlan_country_info_fnm",StringType(),True) \
	.add("wlan_country_info_code",StringType(),True) \
	.add("wlan_da",StringType(),True) \
	.add("wlan_fc_ds",StringType(),True) \
	.add("wlan_fc_frag",IntegerType(),True) \
	.add("wlan_fc_order",IntegerType(),True) \
	.add("wlan_fc_moredata",IntegerType(),True) \
	.add("wlan_fc_protected",IntegerType(),True) \
	.add("wlan_fc_pwrmgt",IntegerType(),True) \
	.add("wlan_fc_type",IntegerType(),True) \
	.add("wlan_fc_retry",IntegerType(),True) \
	.add("wlan_fc_subtype",IntegerType(),True) \
	.add("wlan_fcs_bad_checksum",IntegerType(),True) \
	.add("wlan_fixed_beacon",IntegerType(),True) \
	.add("wlan_fixed_capabilities_ess",IntegerType(),True) \
	.add("wlan_fixed_capabilities_ibss",IntegerType(),True) \
	.add("wlan_fixed_reason_code",IntegerType(),True) \
	.add("wlan_fixed_timestamp",DoubleType(),True) \
	.add("wlan_ra",StringType(),True) \
	.add("wlan_radio_duration",IntegerType(),False) \
	.add("wlan_rsn_ie_gtk_key",IntegerType(),True) \
	.add("wlan_rsn_ie_igtk_key",IntegerType(),True) \
	.add("wlan_rsn_ie_pmkid",IntegerType(),True) \
	.add("wlan_sa",StringType(),True) \
	.add("wlan_seq",DoubleType(),True) \
	.add("wlan_ssid",StringType(),True) \
	.add("wlan_ta",StringType(),True) \
	.add("wlan_tag",IntegerType(),True) \
	.add("wlan_tag_length",StringType(),True) \
	.add("wlan_radio_channel",IntegerType(),True) \
	.add("wlan_radio_data_rate",DoubleType(),True) \
	.add("wlan_radio_end_tsf",DoubleType(),True) \
	.add("wlan_radio_frequency",IntegerType(),True) \
	.add("wlan_radio_signal_dbm",IntegerType(),True) \
	.add("wlan_radio_start_tsf",DoubleType(),True) \
	.add("wlan_radio_phy",IntegerType(),True) \
	.add("wlan_radio_timestamp",DoubleType(),True) \
	.add("wlan_rsn_capabilities_mfpc",DoubleType(),True) \
	.add("wlan_rsna_eapol_keydes_msgnr",DoubleType(),True) \
	.add("wlan_rsna_eapol_keydes_data",StringType(),True) \
	.add("wlan_rsna_eapol_keydes_data_len",DoubleType(),True) \
	.add("wlan_rsna_eapol_keydes_key_info_key_mic",DoubleType(),True) \
	.add("wlan_rsna_eapol_keydes_nonce",StringType(),True) \
	.add("eapol_keydes_key_len",DoubleType(),True) \
	.add("eapol_keydes_replay_counter",DoubleType(),True) \
	.add("eapol_len",DoubleType(),True) \
	.add("eapol_type",DoubleType(),True) \
	.add("llc",StringType(),True) \
	.add("arp",StringType(),True) \
	.add("arp_hw_type",DoubleType(),True) \
	.add("arp_proto_type",StringType(),True) \
	.add("arp_hw_size",DoubleType(),True) \
	.add("arp_proto_size",DoubleType(),True) \
	.add("arp_opcode",DoubleType(),True) \
	.add("arp_src_hw_mac",StringType(),True) \
	.add("arp_src_proto_ipv4",StringType(),True) \
	.add("arp_dst_hw_mac",StringType(),True) \
	.add("arp_dst_proto_ipv4",StringType(),True) \
	.add("ip_dst",StringType(),True) \
	.add("ip_proto",IntegerType(),True) \
	.add("ip_src",StringType(),True) \
	.add("ip_ttl",IntegerType(),True) \
	.add("ip_version",IntegerType(),True) \
	.add("data_data",StringType(),True) \
	.add("data_len",IntegerType(),True) \
	.add("icmpv6_mldr_nb_mcast_records",DoubleType(),True) \
	.add("icmpv6_ni_nonce",IntegerType(),True) \
	.add("tcp_ack",IntegerType(),True) \
	.add("tcp_ack_raw",IntegerType(),True) \
	.add("tcp_analysis",IntegerType(),True) \
	.add("tcp_analysis_flags",IntegerType(),True) \
	.add("tcp_analysis_retransmission",IntegerType(),True) \
	.add("tcp_analysis_reused_ports",IntegerType(),True) \
	.add("tcp_analysis_rto_frame",IntegerType(),True) \
	.add("tcp_checksum",StringType(),True) \
	.add("tcp_checksum_status",IntegerType(),True) \
	.add("tcp_flags_syn",IntegerType(),True) \
	.add("tcp_dstport",IntegerType(),True) \
	.add("tcp_flags_ack",IntegerType(),True) \
	.add("tcp_flags_fin",IntegerType(),True) \
	.add("tcp_flags_push",IntegerType(),True) \
	.add("tcp_flags_reset",IntegerType(),True) \
	.add("tcp_option_len",IntegerType(),True) \
	.add("tcp_payload",StringType(),True) \
	.add("tcp_seq",IntegerType(),True) \
	.add("tcp_seq_raw",IntegerType(),True) \
	.add("tcp_srcport",IntegerType(),True) \
	.add("tcp_time_delta",DoubleType(),True) \
	.add("tcp_time_relative",DoubleType(),True) \
	.add("udp_dstport",IntegerType(),True) \
	.add("udp_srcport",IntegerType(),True) \
	.add("udp_length",IntegerType(),True) \
	.add("udp_payload",StringType(),True) \
	.add("udp_time_relative",DoubleType(),True) \
	.add("udp_time_delta",DoubleType(),True) \
	.add("nbns",StringType(),True) \
	.add("nbss_continuation_data",StringType(),True) \
	.add("nbss_type",StringType(),True) \
	.add("nbss_length",DoubleType(),True) \
	.add("ldap",StringType(),True) \
	.add("smb_access_generic_execute",DoubleType(),True) \
	.add("smb_access_generic_read",DoubleType(),True) \
	.add("smb_access_generic_write",DoubleType(),True) \
	.add("smb_flags_notify",DoubleType(),True) \
	.add("smb_flags_response",DoubleType(),True) \
	.add("smb_flags2_nt_error",DoubleType(),True) \
	.add("smb_flags2_sec_sig",DoubleType(),True) \
	.add("smb_mid",DoubleType(),True) \
	.add("smb_nt_status",DoubleType(),True) \
	.add("smb_server_component",StringType(),True) \
	.add("smb_pid_high",DoubleType(),True) \
	.add("smb_tid",DoubleType(),True) \
	.add("smb2_acct",StringType(),True) \
	.add("smb2_auth_frame",StringType(),True) \
	.add("smb2_buffer_code",StringType(),True) \
	.add("smb2_cmd",StringType(),True) \
	.add("smb2_data_offset",StringType(),True) \
	.add("smb2_domain",StringType(),True) \
	.add("smb2_fid",StringType(),True) \
	.add("smb2_filename",StringType(),True) \
	.add("smb2_header_len",StringType(),True) \
	.add("smb2_host",StringType(),True) \
	.add("smb2_msg_id",StringType(),True) \
	.add("smb2_pid",StringType(),True) \
	.add("smb2_previous_sesid",StringType(),True) \
	.add("smb2_protocol_id",StringType(),True) \
	.add("smb2_sesid",StringType(),True) \
	.add("smb2_session_flags",StringType(),True) \
	.add("smb2_tid",StringType(),True) \
	.add("smb2_write_length",IntegerType(),True) \
	.add("dhcp",StringType(),True) \
	.add("dhcp_client_id_duid_ll_hw_type",StringType(),True) \
	.add("dhcp_cookie",StringType(),True) \
	.add("dhcp_hw_addr_padding",DoubleType(),True) \
	.add("dhcp_hw_mac_addr",StringType(),True) \
	.add("dhcp_id",StringType(),True) \
	.add("dhcp_ip_client",StringType(),True) \
	.add("dhcp_ip_relay",StringType(),True) \
	.add("dhcp_ip_server",StringType(),True) \
	.add("dhcp_option_broadcast_address",StringType(),True) \
	.add("dhcp_option_dhcp_server_id",StringType(),True) \
	.add("dhcp_option_router",StringType(),True) \
	.add("dhcp_option_vendor_bsdp_message_type",StringType(),True) \
	.add("mdns",StringType(),True) \
	.add("dns",StringType(),True) \
	.add("dns_a",StringType(),True) \
	.add("dns_count_add_rr",IntegerType(),True) \
	.add("dns_count_answers",IntegerType(),True) \
	.add("dns_count_auth_rr",IntegerType(),True) \
	.add("dns_count_labels",IntegerType(),True) \
	.add("dns_count_queries",IntegerType(),True) \
	.add("dns_flags_authoritative",StringType(),True) \
	.add("dns_flags_checkdisable",StringType(),True) \
	.add("dns_flags_opcode",StringType(),True) \
	.add("dns_flags_response",StringType(),True) \
	.add("dns_id",StringType(),True) \
	.add("dns_ptr_domain_name",StringType(),True) \
	.add("dns_qry_name",StringType(),True) \
	.add("dns_qry_name_len",IntegerType(),True) \
	.add("dns_resp_len",StringType(),True) \
	.add("dns_resp_name",StringType(),True) \
	.add("dns_resp_ttl",StringType(),True) \
	.add("dns_resp_len_1",StringType(),True) \
	.add("dns_retransmit_request",IntegerType(),True) \
	.add("dns_retransmit_response",IntegerType(),True) \
	.add("dns_time",DoubleType(),True) \
	.add("ssdp",StringType(),True) \
	.add("http_connection",StringType(),True) \
	.add("http_content_length",DoubleType(),True) \
	.add("http_content_type",StringType(),True) \
	.add("http_date",StringType(),True) \
	.add("http_file_data",StringType(),True) \
	.add("http_host",StringType(),True) \
	.add("http_last_modified",StringType(),True) \
	.add("http_location",StringType(),True) \
	.add("http_next_request_in",StringType(),True) \
	.add("http_next_response_in",StringType(),True) \
	.add("http_request_full_uri",StringType(),True) \
	.add("http_request_line",StringType(),True) \
	.add("http_request_method",StringType(),True) \
	.add("http_request_uri_path",StringType(),True) \
	.add("http_request_uri_query",StringType(),True) \
	.add("http_request_uri_query_parameter",StringType(),True) \
	.add("http_request_version",StringType(),True) \
	.add("http_request_in",DoubleType(),True) \
	.add("http_response_code",DoubleType(),True) \
	.add("http_response_code_desc",StringType(),True) \
	.add("http_response_line",StringType(),True) \
	.add("http_response_phrase",StringType(),True) \
	.add("http_response_version",StringType(),True) \
	.add("http_response_for_uri",StringType(),True) \
	.add("http_response_in",StringType(),True) \
	.add("http_referer",StringType(),True) \
	.add("http_time",DoubleType(),True) \
	.add("http_server",StringType(),True) \
	.add("json_value_string",StringType(),True) \
	.add("json_key",StringType(),True) \
	.add("ssh_cookie",StringType(),True) \
	.add("ssh_compression_algorithms_client_to_server_length",IntegerType(),True) \
	.add("ssh_compression_algorithms_server_to_client_length",IntegerType(),True) \
	.add("ssh_direction",IntegerType(),True) \
	.add("ssh_dh_gex_max",IntegerType(),True) \
	.add("ssh_dh_gex_min",IntegerType(),True) \
	.add("ssh_dh_gex_nbits",IntegerType(),True) \
	.add("ssh_encryption_algorithms_client_to_server_length",IntegerType(),True) \
	.add("ssh_encryption_algorithms_server_to_client_length",IntegerType(),True) \
	.add("ssh_host_key_length",IntegerType(),True) \
	.add("ssh_host_key_type_length",IntegerType(),True) \
	.add("ssh_kex_algorithms_length",IntegerType(),True) \
	.add("ssh_mac_algorithms_client_to_server_length",IntegerType(),True) \
	.add("ssh_mac_algorithms_server_to_client_length",IntegerType(),True) \
	.add("ssh_message_code",StringType(),True) \
	.add("ssh_mpint_length",StringType(),True) \
	.add("ssh_packet_length",IntegerType(),True) \
	.add("ssh_packet_length_encrypted",StringType(),True) \
	.add("ssh_padding_length",IntegerType(),True) \
	.add("ssh_padding_string",IntegerType(),True) \
	.add("ssh_protocol",StringType(),True) \
	.add("ssh_server_host_key_algorithms_length",IntegerType(),True) \
	.add("tls_alert_message_desc",IntegerType(),True) \
	.add("tls_alert_message_level",IntegerType(),True) \
	.add("tls_app_data_proto",StringType(),True) \
	.add("tls_compress_certificate_compressed_certificate_message_length",DoubleType(),True) \
	.add("tls_connection_id",StringType(),True) \
	.add("tls_handshake_extension_type",StringType(),True) \
	.add("tls_handshake_extensions_key_share_group",StringType(),True) \
	.add("tls_handshake_session_ticket_length",IntegerType(),True) \
	.add("tls_handshake_version",StringType(),True) \
	.add("tls_record_content_type",IntegerType(),True) \
	.add("tls_record_version",StringType(),True) \
	.add("StringLabel",StringType(),False)








we read all the csv files into a dataframe and print the schema to check it looks correct

In [58]:
df = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load(analysisFile)

df.printSchema()

root
 |-- frame_encap_type: integer (nullable = true)
 |-- frame_len: integer (nullable = true)
 |-- frame_number: integer (nullable = true)
 |-- frame_time: string (nullable = true)
 |-- frame_time_delta: double (nullable = true)
 |-- frame_time_delta_displayed: double (nullable = true)
 |-- frame_time_epoch: double (nullable = true)
 |-- frame_time_relative: double (nullable = true)
 |-- radiotap_channel_flags_cck: integer (nullable = true)
 |-- radiotap_channel_flags_ofdm: integer (nullable = true)
 |-- radiotap_channel_freq: integer (nullable = true)
 |-- radiotap_datarate: double (nullable = true)
 |-- radiotap_dbm_antsignal: integer (nullable = true)
 |-- radiotap_length: integer (nullable = true)
 |-- radiotap_mactime: double (nullable = true)
 |-- radiotap_present_tsft: string (nullable = true)
 |-- radiotap_rxflags: string (nullable = true)
 |-- radiotap_timestamp_ts: long (nullable = true)
 |-- radiotap_vendor_oui: integer (nullable = true)
 |-- wlan_duration: integer (nullab

look at the session, so we can see how the job was exectuted in the Spark UI

In [59]:
SparkSession.getActiveSession()

Spark best practice is to create our parquet files with no null values, we replace the numberical values with a 0. Usually a string null would be and empty string, however one hot encoder can not handle these so we replace with a NA instead

In [60]:

df_removeNullNumbers=df.fillna(0)


In [61]:
df_no_null=df_removeNullNumbers.fillna("NA")

we want to have a binary coding for the label, Spark MLib needs a numeric label so we create that here.

In [63]:
from pyspark.sql.functions import when
from pyspark.sql.functions import lit
df_with_binary_label = df_no_null.withColumn("label",\
   when(df_no_null.StringLabel == "Normal", lit(0)) \
     .otherwise(lit(1)) \
  )

We then right the dataframe as a Parquet file.This is a columnar dataformat that allows faster processing. We count the number of rows then read the file back. The numbers should be the same and of they are we have successfully created our Parquet file for analysis.

https://spark.apache.org/docs/latest/sql-data-sources-parquet.html

In [64]:
df_with_binary_label.write.parquet(writePath)

In [65]:
df_with_binary_label.count()

3778706

In [66]:

parquetFile = spark.read.parquet(writePath)

In [67]:
parquetFile.count()

3778706

Problems with the Data

The SSDP attack has more issues than the other attacks with some missing labels and blank rows and some mislabled SDDP instead of SSDP I cleaned these with some extra cells not on this notebook. The numbers is NA were small although it did seem to be affected/affecting the line above. I will need to spend extra attention on this file as it is also one of the biggest and has a high percentage of attack packets. 

Some of the Kr00k attacks were similarly labeled with a capital K at the end I adjusted the SSDP script to clean this. As an FYI here is the code
***********************************************************************************************************************************************8
from pyspark.sql.functions import when
df2=df_no_null.withColumn("StringLabel", when(df_no_null.StringLabel == "Kr00K","Kr00k") \
      .otherwise(df_no_null.StringLabel))

The botnet attack appears to be missing the start of the attack, it starts at Frame 10,000